<a href="https://colab.research.google.com/github/Lesiazhou/QM2/blob/main/S2_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing required libraries

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pylab
%matplotlib inline
pylab.rcParams['figure.figsize'] = (8, 8)

# Convert to dataframe

In [ ]:
df = pd.read_csv('https://storage.googleapis.com/sample-bucket-qm2/ndvi.csv').drop(columns=['Unnamed: 0'])
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,NDVI,Latitude,Longitude,Region
0,2000-02-18,-0.0134,46.182852,60.758907,North
1,2000-02-18,-0.0145,46.182852,60.765402,North
2,2000-02-18,-0.0139,46.182852,60.771897,North
3,2000-02-18,-0.0141,46.182852,60.778391,North
4,2000-02-18,-0.0117,46.182852,60.784886,North
...,...,...,...,...,...
1245207,2023-02-02,0.0138,46.011982,60.823428,South
1245208,2023-02-02,0.0203,46.011982,60.829903,South
1245209,2023-02-02,0.0182,46.007485,60.786114,South
1245210,2023-02-02,0.0165,46.007485,60.792588,South


# Spatial Regression Discontinuity

In [ ]:
df['Year'] = df['Date'].dt.year

In [ ]:
df['Distance'] = np.sqrt((df['Latitude']-46.09667)**2 + (df['Longitude']-60.79306)**2)

In [ ]:
pre = df[(df['Year']<2005) & (df['Region']=='North')]
pre = pre.groupby('Distance')['NDVI'].mean().reset_index()
pre

,Distance,NDVI
0,0.001527,0.074600
1,0.001527,0.067541
2,0.005206,0.065129
3,0.005206,0.068100
4,0.005247,0.071891
...,...,...
1266,0.127283,0.072300
1267,0.128361,0.084734
1268,0.128361,0.094000
1269,0.128361,0.219500


In [ ]:
post = df[(df['Year']>=2005) & (df['Region']=='North')]
post = post.groupby('Distance')['NDVI'].mean().reset_index()
post

,Distance,NDVI
0,0.001527,0.109477
1,0.001527,0.134528
2,0.001527,0.040008
3,0.002178,0.126760
4,0.002834,0.125800
...,...,...
3263,0.128361,0.064394
3264,0.128361,0.152853
3265,0.128361,0.161222
3266,0.128361,0.145798


In [ ]:
changes = pd.merge(pre, post, on='Distance')
changes['Change'] = changes['NDVI_y'] - changes['NDVI_x']
changes.drop(columns=['NDVI_x', 'NDVI_y'], inplace=True)
changes

,Distance,Change
0,0.001527,0.034877
1,0.001527,-0.027532
2,0.005206,-0.016477
3,0.005206,0.115393
4,0.005247,0.071065
...,...,...
1266,0.127283,0.004777
1267,0.128361,-0.033210
1268,0.128361,-0.029606
1269,0.128361,-0.066647


In [ ]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

model= ols('Change ~  Distance', data=changes).fit() # fit the model
print(model.summary()) # print the summary

                            OLS Regression Results                            
Dep. Variable:                 Change   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     78.08
Date:                Mon, 06 Jan 2025   Prob (F-statistic):           3.23e-18
Time:                        14:40:43   Log-Likelihood:                 1271.5
No. Observations:                1271   AIC:                            -2539.
Df Residuals:                    1269   BIC:                            -2529.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0852      0.007     12.232      0.0